In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import pickle
from tqdm import tqdm

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
frag_train = pd.read_csv('../data/all_train_aligned.csv')
frag_dev = pd.read_csv('../data/all_dev_aligned.csv')
frag_test = pd.read_csv('../data/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

In [ ]:
with open('../data/freq/freq_token_db.pkl', 'rb') as f:
  words_and_levels = pickle.load(f)

In [ ]:
def get_rl(token, model, oov_level = 0):
    try:
        return model[token]
    except:
        return oov_level

def levels_inference_pipeline(fragment, model, v = False):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  levels = [get_rl(token, model, 0) for token in tokens]
  levels = [a if a > 3 else 3 for a in levels]
  if v:
    print(tokens)
    print(levels)

  return levels

In [1]:
def get_gt_levels(fragment):
  return [int(t.split('#')[1]) for t in fragment.split(' ')]

In [ ]:
gt_levels = [get_gt_levels(f) for f in frag_test['0']]

In [ ]:
rr = [levels_inference_pipeline(a, words_and_levels, v = True) for a in frag_test['0'][99:105]]


['وقد', 'يكون', 'هذا', 'من', 'ذنوبي', '.']
[3, 3, 3, 3, 4, 3]
['ولكن', 'من', 'المعاملة', 'ما', 'هو', 'ظلم', '،']
[3, 3, 3, 3, 3, 3, 3]
['وقد', 'تكون', 'معاملتك', 'إياي', 'من', 'هذا', 'القبيل', '.']
[3, 3, 5, 4, 3, 3, 3, 3]
['رجل', 'صريح', 'لم', 'يألف', 'المكاتمة', 'يجهر', 'برأيه', 'فيعد', 'من', 'أجل', 'ذلك', 'سيئ', 'الأدب', '!']
[3, 3, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3]
['فقالت', 'ووجهها', 'إلى', 'النافذة', ':', 'لست', 'أسمح', 'للأغراب', 'أن', 'يجترئوا', 'علي', 'حتى', 'بالمديح', '.']
[3, 4, 3, 3, 3, 3, 4, 5, 3, 5, 3, 3, 5, 3]
['فقال', 'بلهجة', 'الظافر', ':', 'آه', '!']
[3, 4, 4, 3, 3, 3]


In [ ]:
predicted_levels_freq = [levels_inference_pipeline(a, words_and_levels) for a in frag_test['0']]

In [ ]:
predicted_levels_freq[99:105]

[[3, 3, 3, 3, 4, 3],
 [3, 3, 3, 3, 3, 3, 3],
 [3, 3, 5, 4, 3, 3, 3, 3],
 [3, 3, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3],
 [3, 4, 3, 3, 3, 3, 4, 5, 3, 5, 3, 3, 5, 3],
 [3, 4, 4, 3, 3, 3]]

In [ ]:
gt_levels[99:105]

[[3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 5, 3, 3, 5, 3],
 [3, 3, 3, 4, 5, 3, 3, 3, 5, 5, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3],
 [3, 4, 4, 3, 3, 3]]

In [ ]:

preds = np.concatenate(predicted_levels_freq)
gt = np.concatenate(gt_levels)

In [ ]:
frag_test['0'][103]

'رجل#3 صريح#3 لم#3 يألف#4 المكاتمة#5 يجهر#3 برأيه#3 فيعد#3 من#5 أجل#5 ذلك#3 سيئ#3 الأدب#3 !#3'

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(gt, preds))

              precision    recall  f1-score   support

           3       0.95      0.82      0.88     23851
           4       0.18      0.28      0.22      2032
           5       0.20      0.60      0.30      1040

    accuracy                           0.77     26923
   macro avg       0.44      0.57      0.47     26923
weighted avg       0.86      0.77      0.81     26923

